# PCA Evaluation

___
This notebook is focused on the evaluation of principal component analysis (PCA).

PCA was used as a tool to study data projections that preserved most of the original data variance, evaluating the features that would have the largest weights, identifying them as the ones most important for such projections.

___